In [93]:
import mbuild as mb
import numpy as np

class CH2(mb.Compound):
    """Creates a monomer for a straight alkyl chain"""
    def __init__(self):
        super(CH2, self).__init__()
        self.add(mb.Particle(name='C', pos=[0,0,0]), label='C[$]')
        self.add(mb.Particle(name='H', pos=[-0.109, 0, 0.0]), label='HC[$]')    
        self.add(mb.Particle(name='H', pos=[0.109, 0, 0.0]), label='HC[$]')
        theta = 0.5 * (180 - 109.5) * np.pi / 180
        self['HC'][0].rotate(theta, around=[0, 1, 0])
        self['HC'][1].rotate(-theta, around=[0, 1, 0])
        self.add_bond((self[0],self['HC'][0]))
        self.add_bond((self[0],self['HC'][1]))
        
        self.add(mb.Port(anchor=self[0]), label='up')
        self['up'].translate([0, -0.154/2, 0]) 
        self['up'].rotate(theta, around=[1, 0, 0])
        self.add(mb.Port(anchor=self[0]), label='down')
        self['down'].translate([0, 0.154/2, 0]) 
        self['down'].rotate(np.pi, [0, 1, 0])
        self['down'].rotate(-theta, around=[1, 0, 0]) 
        
class CH3(mb.Compound):
    """Creates a methyl group"""
    def __init__(self):
        super(CH3, self).__init__()
        self.add(mb.Particle(name='C', pos=[0,0,0]), label='C[$]')
        self.add(mb.Particle(name='H', pos=[-0.109, 0, 0.0]), label='HC[$]')    
        self.add(mb.Particle(name='H', pos=[0.109, 0, 0.0]), label='HC[$]')
        self.add(mb.Particle(name='H', pos=[0, 0.154, 0]), label='HC[$]')
        theta = 0.5 * (180 - 109.5) * np.pi / 180
        self['HC'][0].rotate(theta, around=[0, 1, 0])
        self['HC'][1].rotate(-theta, around=[0, 1, 0])
        self['HC'][2].rotate(-theta, around=[1, 0, 0])
        self.add_bond((self[0],self['HC'][0]))
        self.add_bond((self[0],self['HC'][1]))
        self.add_bond((self[0],self['HC'][2]))
        
        self.add(mb.Port(anchor=self[0]), label='chain')
        self['chain'].translate([0, -0.154/2, 0]) 
        self['chain'].rotate(theta, around=[1, 0, 0])
        
class OH(mb.Compound):
    """Creates a hydroxyl group"""
    def __init__(self):
        super(OH, self).__init__()
        self.add(mb.Particle(name='O', pos=[0,0,0]), label='O')
        self.add(mb.Particle(name='H', pos =[.096, 0, 0]), label='H')
        self.add_bond((self[0], self[1]))
        
        self.add(mb.Port(anchor=self[0]), label='cbond')
        self['cbond'].translate([-.143/2, 0, 0])
        theta = (180 - 104.5) * np.pi / 180
        self['cbond'].rotate(theta, around=[0,1,0])


In [95]:
#use base classes to constuct a new alcohol class


class Alcohol(mb.Compound):
    """Creates an n-alcohol based on user input"""
    def __init__(self, chain_length):
        super(Alcohol, self).__init__()
        self.add(CH3(), label="tailcap")
        tail = mb.Polymer(CH2(), (chain_length - 1))
        self.add(tail, label="tail")
        self.add(OH(), label="head")
        mb.force_overlap(move_this=self['tailcap'],from_positions=self['tailcap']['chain'],
                         to_positions=self['tail']['up'])
        mb.force_overlap(move_this=hydroxyl, from_positions=self['head']['cbond'], 
                         to_positions=self['tail']['down'])

#instantiation and visualization
alcohol = Alcohol(6)
alcohol.energy_minimization()
alcohol.visualize()

/home/loganguy/builds/mbuild/mbuild/compound.py:1240: UserWarning: Performing energy minimization using the Open Babel package. Please refer to the documentation to find the appropriate citations for Open Babel and the UFF force field
  "Open Babel and the {} force field".format(forcefield))
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2108: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/pandas/io/parsers.py:2110: FutureWarning: split() requires a non-empty pattern match.
  yield pat.split(line.strip())
/home/loganguy/anaconda3/envs/mosdef/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:464: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  self.log.warn(message)
The installed widget Javascript is the wrong version.


In [96]:
alcohol.save('alcohol.mol2', overwrite=True)